# LLM Zoomcamp - Homework 01
Emanuel Tomé\
June 2024

## Q1. Running Elastic

# Answer: 42f05b9372a9a4a470db3b52817899b99a76ee73

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

## Q3. Searching

## Q4. Filtering

## Q5. Building a prompt

## Q6. Tokens

## Bonus: generating the answer